## 8.ResNet(2015年)

**学习目标**

1. 能够构建ResNet模型中的残差块

2. 能够构建RseNet18模型

3. 理解模型中的残差结构为什么能显著改善网络性能

4. 理解1x1卷积核的作用

****

### 8.1 ResNet模型

1. **ResNet模型概述**

​		2015 年，152 层深的 ResNet 横空出世，不仅取得当年ImageNet竞赛冠军，相关论文在CVPR 2016斩获最佳论文奖，并深深的影响了后来的深度神经⽹络的设计。自那以后，许多研究对 ResNet 架构进行了调整和改进，其中最著名的有获得CVPR 2017 Best Paper的DenseNet模型，谷歌的MobileNet系列和EfficientNet系列。

​		深度残差网络（deep Residual Network）是近些年来计算机视觉和深度学习领域最具突破性的工作之一。ResNet 使得训练深达数百甚至数千层的网络成为可能，而且性能仍然优异。由于其表征能力强，ResNet 在图像分类任务之外的许多计算机视觉应用上也取得了巨大的性能提升，例如对象检测和人脸识别。

​		根据通用近似定理（universal approximation theorem），如果给定足够的容量，一个单层的前馈网络就足以表示任何函数。但是，这个层可能是非常大的，而且网络容易过拟合数据。因此，研究界有一个共同的趋势，就是网络架构需要更深。

​		从 AlexNet 提出以来，SOTA（state-of-the art） 的 CNN 架构都是越来越深。虽然 AlexNet 只有5层卷积层，但后来的 VGG 网络和 GoogLeNet分别有19层和22层。但是，如果只是简单地将层堆叠在一起，增加网络的深度并不会起太大作用。这是由于难搞的梯度消失（vanishing gradient）问题，深层的网络很难训练。因为梯度反向传播到前一层，重复相乘可能使梯度无穷小。结果就是，随着网络的层数更深，其性能趋于饱和，甚至开始迅速下降。

​		在 ResNet出现之前，已经有好几种处理梯度消失问题的方法，但遗憾的是没有一个方法能够真正解决这个问题。另外，ResNet也不是第一个利用 shortcut connection 的方法，R. Srivastava 等人提出的 Highway Network就引入了门控短路连接（gated shortcut connections）。这些参数化的门控制允许多少信息流过短路通道（shortcut）。类似的想法可以在长短期记忆网络（LSTM）单元中找到，其中存在一个参数化的“遗忘门”（forget gate），它控制让多少信息传送到下一个时间步长。因此，ResNet 被认为是 Highway Networks 的一种特殊情况。

​		然而实验表明，Highway Network 的性能不如 ResNet，因为 Highway Network 的解决方案空间包含 ResNet，它至少应该表现得跟 ResNet 一样好才对，这就显得有点奇怪。这表明，保持这些gradient highways的干净相比追求更大的解决方案空间更为重要。按照这种直觉，何恺明等人改进了残差块，并提出一个残差预激活变体，其中梯度可以畅通无阻地通过 shortcut 连接到任何其他先前的层。

2. **ResNet的作者**

​		ResNet网络是由微软亚洲研究院孙剑团队完成的。论文作者分别是何恺明、张祥雨、任少卿和孙剑。ResNet的第一作者何恺明本科就读于清华大学，博士毕业于香港中文大学多媒体实验室(MMLab)，导师为著名人工智能科学家汤晓鸥教授。2011年加入微软亚洲研究院（MSRA）工作，主要研究计算机视觉和深度学习。2015年的ImageNet图像识别大赛中，何恺明用ResNet击败了谷歌、英特尔、高通等业界团队，荣获第一。2016年，加入Facebook AI Research（FAIR）担任研究科学家。2024正式加入MIT担任副教授。

学术荣誉：

2009年，首位获得CVPR最佳论文奖的中国学者。

2016年，CVPR最佳论文奖。

2017年，ICCV Marr Prize和最佳学生论文奖，香港中文大学工程学院杰出校友。

2018年，CVPR PAMI年轻学者奖。

2022年，荣登人工智能全球最具影响力学者榜单第一名。

<img src="./images/Kaiming.png" style="zoom:60%;" />

3. **ResNet网络在解决什么问题**

​		模型深度对性能的影响至关重要，当增加网络层数后，网络可以进行更加复杂的特征模式的提取，所以当模型更深时理论上可以取得更好的结果。但是更深的网络其性能一定会更好吗？

​		实验发现网络构建的越来越深时，训练模型会出现模型退化现象。模型退化现象是指在没有梯度消失或梯度爆炸，模型能够收敛的情况下，网络越深，模型的准确率越低，同时，模型的准确率先达到饱和，此后迅速下降。如下图所示：

<img src="./images/3-1.png" style="zoom:68%;" />

​		为什么会存在模型退化现象？一种解释是说神经网络越来越深的时候，反传回来的梯度之间的相关性会越来越差，最后接近白噪声。因为我们知道图像是具备局部相关性的，那其实可以认为梯度也应该具备类似的相关性，这样更新的梯度才有意义，如果梯度接近白噪声，那梯度更新可能根本就是在做随机扰动。有了梯度相关性这个指标之后，通过对一系列网络结构和激活函数的分析，发现ResNet在保持梯度相关性方面性能优异。

​		ResNet的核⼼思想是每个附加层都应该更容易地包含原始函数作为其元素之⼀。ResNet网络模型最重要的结构就是残差块的构建，下面的内容中，我们首先构造残差块，然后利用残差块来实现ResNet18模型。

### 8.2  残差块的构建

1. **什么是残差块？**

​		如下图所⽰，假设我们的原始输⼊为 x ，希望学到的理想映射为 f(x)。左图虚线框中的部分需要直接拟合出该映射 f(x) ，而右图虚线框中的部分则需要拟合出残差映射 f(x) − x 。残差映射在现实中往往更容易优化。假如我们希望学出的理想映射 f(x)为恒等映射 ，只需将下图右边虚线框上⽅的加权运算的权重和偏置参数设成 0，那么 f(x) 即为恒等映射。当理想映射 f(x) 极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。右图是 ResNet 的基础结构‒残差块（residual block）。在残差块中，输⼊可通过跨层数据通路（shortcut）更快地向前传播。

<img src="./images/residualblock.png" style="zoom:60%;" />



2. **残差块的结构**

​		ResNet的残差块⾥⾸先有 2 个有相同输出通道数的 3 × 3 卷积层。每个卷积层后接⼀个批量标准化层和 ReLU 激活函数。然后我们通过跨层数据通路，跳过这 2 个卷积运算，将输⼊直接加在最后的 ReLU 激活函数前。这样的设计要求 2 个卷积层的输出与输⼊形状⼀样，从而可以相加。如果想改变通道数和特征维度（特征图的高和宽），就需要引⼊⼀个额外的 1 × 1 卷积层来将输⼊变换成需要的通道数和特征维度后再做相加运算。

<img src="./images/residual.png" style="zoom:60%;" />



3. **残差块的实现**

​		首先，我们导入必需的模块。torch.nn模块是由PyTorch提供帮助开发人员很方便的创建和训练神经网络的模块。它的主要功能就是创建和训练神经网络。nn.functional.x是函数接口，而nn.X是nn.functional.x的类封装，并且nn.X都继承于一个共同祖先nn.Module。这一点导致nn.X除了具有nn.functional.x功能之外，内部附带了nn.Module相关的属性和方法，例如train(), eval(),load_state_dict, state_dict 等。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

​		然后，我们来构建一个实现残差块的ResidualBlock类。在ResidualBlock类的初始化方法中，我们定义了5个内部变量conv1、bn1、conv2、bn2和shortcut。其中shortcut为跨层数据通路，我们通过判断残差块的输入和输出通道数是否相等或者第一个卷积核的步长是否为1来确定是否要在跨层数据通路shortcut上加入1x1的卷积核。在forward方法中，我们定义了整个残差块前向传播信号的传输路径，从而构建了符合要求的残差块。

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=strides, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        
        if in_channels != out_channels or strides != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=strides),
                nn.BatchNorm2d(out_channels))

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.shortcut:
            x = self.shortcut(x)
        out += x

        return F.relu(out)

在构建好ResidualBlock类之后，我们就可以使用残差块来实现ResNet 18模型了。

### 8.3  ResNet18模型的构建

1.  **ResNet18模型**

​		如下图所示的ResNet模型，前两层在输出通道数为 64、步幅为 2 的 7 *×* 7 卷积层后，接步幅为 2 的 3 *×* 3 的最⼤池化层。每个卷积层后为批量归⼀化层，使⽤ 4 个由残差块组成的模块，每个模块使⽤2个同样输出通道数的残差块。第⼀个模块的通道数同输⼊通道数⼀致。由于之前已经使⽤了步幅为2 的最⼤池化层，所以⽆须减小⾼和宽。之后的每个模块在第⼀个残差块⾥将上⼀个模块的通道数翻倍，并将⾼和宽减半。接着在模型中加⼊所有模块。最后，在模型中加⼊自适应平均池化层，以及全连接层输出。每个模块有 4 个卷积层（不包括恒等映射的 1 *×* 1 卷积层），加上第⼀个 7 *×* 7 卷积层和最后⼀个全连接层，一共 18 层。这就是为什么我们称它为 ResNet18的原因。

<img src="./images/resnet18.png" style="zoom:70%;" />

​		通过配置不同的通道数和模块⾥的残差块数可以得到不同的 ResNet 模型，例如更深的含152层的 ResNet-152。ResNet结构简单，修改⽅便。这些因素都导致了 ResNet 迅速被⼴泛使⽤。

2. **resnet_block模块的搭建**

​		我们通过观察上图的网络结构可以发现，每个模块都是由两个残差块构成的，因此我们首先实现模块。模块的构建是用resnet_block函数实现的，函数的参数包括in_channels, out_channels, num_residuals, first_block。在函数内部，我们定义了名为block_list的列表，用于保存残差块。每个resnet_block都由两个残差块组成，因此num_residuals的值为2。除了第一个resnet_block外，其他的resnet_block的第一个卷积核的步长都为2，也就意味着这个地方进行了下采样，特征图的宽高减半，所以这里的跨层数据通路需要加入1x1的卷积来做同样的事情。第二个残差块特征图的维度和通道数保持不变。

In [3]:
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    block_list = []

    for i in range(num_residuals):
        if i==0 and not first_block:
            block_list.append(ResidualBlock(in_channels, out_channels, strides=2))
        else:
            block_list.append(ResidualBlock(out_channels, out_channels))

    return block_list

3. **ResNet18模型的构建**

​		ResNet18的第⼀个模块做了特别处理，它是先用一个尺寸为7的卷积核，在批量标准化层和非线性激活层之后。接着在 ResNet 加⼊4个resnet_block块，每个resnet_block块使⽤了 2 个残差块。最后，在 ResNet 中加⼊自适应平均池化层，以及全连接层输出。

In [4]:
def resnet18(num_classes=10, gray=False):
    num_channels = 3
    if gray == True:
        num_channels = 1

    model = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=64, kernel_size=7, stride=2, padding=3), 
                        nn.BatchNorm2d(64),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                        *resnet_block(64, 64, 2, first_block=True),
                        *resnet_block(64, 128, 2),
                        *resnet_block(128, 256, 2),
                        *resnet_block(256, 512, 2),
                        nn.AdaptiveAvgPool2d((1, 1)),
                        nn.Flatten(),
                        nn.Linear(512, num_classes))
    
    return model

4. **观察ResNet18模型结构**

​		在训练 ResNet18模型之前，我们先来观察⼀下ResNet中不同模块的输⼊特征图形状是如何变化的。

In [5]:
model = resnet18()
print(model)
X = torch.randn(1, 3, 224, 224)
print('\n Output shape:')
for layer in model:
    X = layer(X)
    print(layer.__class__.__name__, ':', X.shape)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): ResidualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (5): ResidualBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

### 8.4 ResNet残差块组件分析

​		在ResNet残差块的结构中，有卷积层、批量归一化、非线性激活函数ReLU和跨层数据通路这些组件，下面就分别来分析它们所起的作用。

1. **残差块中间为什么需要激活函数？**

​		从下图可以看出，如果残差块中间没有非线性激活函数的话，整个模块等价于一个线性变换，直接连接到输出的跳线就变得毫无意义。

<img src="./images/28.png" style="zoom:60%;" />

2. **1x1卷积的作用**

残差块的输入和输出的特征图可能会存在维度和通道数量上的变化，跨层数据通路直接从输入连接到输出的话，如果维度和通道数量不匹配，就没有办法进行相加运算。而1x1的卷积可以起到改变通道数量和特征图维度的作用，使得输入和输出特征图能够匹配从而相加。

### 8.5 小结

1. 学习嵌套函数（nested function）是训练神经⽹络的理想情况。在深层神经⽹络中，学习另⼀层作为恒等映射（identity function）较容易（尽管这是⼀个极端情况）。
2. 残差映射可以更容易地学习同⼀函数，例如将权重层中的参数近似为零。
3. 利⽤残差块（residual blocks）可以训练出⼀个有效的深层神经⽹络：输⼊可以通过层间的残余连接更快地向前传播。
4. 残差⽹络（ResNet）对随后的深层神经⽹络设计产⽣了深远影响，⽆论是卷积类⽹络还是全连接类⽹络。